### MOINDRES CARRES ORDINAIRES

*Source: cette série d'exercices s'inpire très largement des tutoriels de Simon Ejdemyr: https://sejdemyr.github.io/r-tutorials/statistics/*

On commence par charger les mêmes données que dans le TD3, avec les mêmes codes strictement (à un petit détail près, cf. note en bas de code):

In [ ]:
#ouverture des données:
library(foreign) #pour ouvrir les données dta de version 14 ou inférieure
anes <- read.dta("TD3_anescum_small_v12.dta")

#on garde seulement quelques variables et on les renomme pour plus de clarté:
library(plyr)
library(dplyr)
anes <- anes %>% 
        dplyr::select(year = VCF0004,
               age = VCF0101,
               gender = VCF0104,
               race = VCF0106,
               edu = VCF0140,
               south = VCF0113,
               income = VCF0114,
               partyid = VCF0301,
               interest = VCF0310,
               govtrust = VCF0604,
               abortion = VCF0838,
               demtherm = VCF0218)
head(anes)

#note: ici, "select" est précédé de "dplyr::". pourquoi ? car nous chargeons aussi le package "arm" qui a aussi sa propre fonction "select" et qui va venir dominer la fonction "select" du package "dplyr". 
#il faut donc préciser de quel package on nécessite la fonction "select"

En plus de **foreign**, **plyr**, **dplyr**, on va avoir besoin des packages : **ggplot2** et un nouveau package: **arm** qui permet de montrer les résultats MCO plus lisiblement et d'extraire les écart-types :

In [ ]:
library(arm)
library(ggplot2)

Rappel : le codebook est disponible : TD3_codebook_anescum_small_v12.txt.

# 1. Modèle de régression simple
Nous allons maintenant estimer un modèle simple par MCO et nous allons en donner une représentation graphique (de plus en plus courant). Un modèle simple est un modèle qui met en relation une variable dépendante y avec une explicative x.

### 1.1 Estimation
Nous souhaitons savoir si les préférences pour les démocrates varient selon le genre. 
Commençons par regarder les variables:

In [ ]:
#on recode gender
levels(anes$gender)
levels(anes$gender) <- c(NA, "Homme", "Femme")
#on inspecte dermtherm et gender
summary(anes$demtherm)
summary(anes$gender)

Nous régressons ensuite *gender* sur *demtherm* (qui varie de 0 à 100), grâce à **lm()** (lm(y~x)).

In [ ]:
#on régresse grâce à lm()
lm1 <- lm(demtherm ~ gender, data = anes)
display(lm1)

### 1.2 Représentation graphique
A partir de cette estimation on peut faire deux types de représentations graphiques: 

- l'effet moyen d'être une femme (relativement à l'homme)
- l'opinion moyenne d'un homme et l'opinion moyenne d'une femme (la différence étant l'effet moyen d'être d'un genre plutôt que d'un autre)

L'option 1 revient à représenter le coefficient de *gender* estimé par MCO. 

L'option 2 revient à représenter la valeur prédite moyenne de l'opinion des hommes d'un côté et des femmes de l'autre.

### 1.2.1  Option 1 : représenter le coefficient

On peut extraire les coefficients avec **coef()**, les écart-types avec **se.coef()**, et l'intervalle de confiance avec **confint()**.

In [ ]:
coef(lm1)

In [ ]:
coef(lm1)[2]

In [ ]:
se.coef(lm1)
se.coef(lm1)[2]

In [ ]:
confint(lm1, level=0.95)[1]

In [ ]:
confint(lm1, level=0.95)

On peut donc les utiiser pour créer un data frame avec toutes les informations nécessaires à contruire un graphique de notre résultat principal. Nous avons besoin: 
- du coefficient de 'gender'
- de l'intervalle de confiance autour de ce coefficient
- que nous devons intégrer un à dataframe

In [ ]:
#coefficient de gender
est1<-coef(lm1)[2]
est1
#intervalle de confiance: 
c1<-confint(lm1,level=0.95)[2,]  #option2
c1
#création du data frame:
est1 <- data.frame(est=est1,
                    c_inf=c1[1],
                    c_sup=c1[2],
                    model="Modèle simple")
#le dataframe crée:
est1
est1[1]

On peut ensuite réaliser le graphique avec **ggplot()**.

In [ ]:
ggplot(est1, aes(x=model,y=est)) +
    geom_point()

On ajoute l'intervalle de confiance :

In [ ]:
ggplot(est1, aes(x=model,y=est)) +
    geom_point() + 
    geom_errorbar(aes(ymin = c_inf, ymax = c_sup), width = 0.1) 
    

On change l'échelle de y pour améliorer la lecture du graphique:

In [ ]:
ggplot(est1, aes(x=model,y=est)) +
    geom_point() + 
    geom_errorbar(aes(ymin = c_inf, ymax = c_sup), width = 0.1) +
    geom_hline(yintercept = 0, lty = 2, color = "red") 

On ajoute des labels aux axes :

In [ ]:
ggplot(est1, aes(x=model,y=est)) +
    geom_point() + 
    geom_errorbar(aes(ymin = c_inf, ymax = c_sup), width = 0.1) +
    geom_hline(yintercept = 0, lty = 2, color = "red") +
    xlab("") +
    ylab("Opinion des femmes sur le parti démocrate (relativement aux hommes)")

### 1.2.2 Option 2 : représenter la moyenne conditionnelle de la variable dépendante y selon les différentes catégories de x

On extrait de l'estimation "lm1" les valeurs prédites du score pro démocrate du thermomètre, ainsi que l'intervalle de confiance autour de ces valeurs prédites. 

In [ ]:
predict_for <- data.frame(gender = c("Homme", "Femme"))

pred1 <- predict(lm1,
                 newdata = predict_for,
                 se.fit = T,
                 interval = "confidence")

Regardons ce que ça donne :

In [ ]:
predict_for
pred1

Ce qui nous intéresse se trouve dans $fit. On crée un nouveau dataframe *pred1* dans leuel on crée la variable *Genre* pour la légende du graphique :

In [ ]:
pred1 <- data.frame(cbind(pred1$fit, predict_for))
pred1

On peut maintenant représenter les coefficients graphiquement par **ggplot** et **geom_point()** :

In [ ]:
ggplot(pred1, aes(x = gender, y = fit, color = gender)) +
    geom_point() 

On ajoute les intervalles de confiance (**geom_errorbar()**) et des labels aux axes (**xlab()** et **ylab()**) : 

In [ ]:

ggplot(pred1, aes(x = gender, y = fit, color = gender)) +
    geom_point() +
    geom_errorbar(aes(ymin = lwr, ymax = upr), width = 0.1) +
    xlab("") +
    ylab("Moyenne du score pro-démocrate (entre 0 et 100)")

Discussion: Que conclure ? Quelle est la relation entre le premier graphique (option 1) et le second graphique (option 2) ?

# 2. Modèle de régression multiple
On peut également estimer un modèle à plusieurs variables explicatives. On ajoute à notre modèle simple deux variables: *income* et *age*.

### 2.1 Estimation

Avant toute chose on vérifie quelle est la nature des données, et on les inspecte pour vérifier que les variables manquantes sont bien encodées NA (not available).

In [ ]:
class(anes$age)
class(anes$income)

In [ ]:
sort(unique(anes$age))
levels(anes$income)

On voit que certaines personnes ont un âge de 0. On voit aussi que la catégorie 1 de *income* correspond en fait à de l'information manquante.

Nous ne souhaitons pas que la régression tienne compte des observations pour lesquelles la valeur est en fait manquante. Pour cela, on effectue les modifications nécessaires concernant les valeurs manquantes : 

In [ ]:
#correction de age
anes$age[anes$age==0]<-NA
sort(unique(anes$age))

#correction des quintiles
levels(anes$income)[1]<-NA
levels(anes$income)

On peut alors régresser :

In [ ]:
lm2 <- lm(demtherm~gender+income+age, data=anes)

Pour visualiser : 

In [ ]:
display(lm2) #infos de base

In [ ]:
summary(lm2) #avec les étoiles

Discussion : interpréter les coefficients estimés par le modèle lm2.

### 2.2 Représentation graphique

**Exercice 1** : 

Réaliser le graphique type 'Option 1' qui montre l'effet du genre sur le thermomètre démocrate, avec le nouveau modèle lm2.

In [ ]:
#voir les objets
coef(lm2)
confint(lm2,level=0.95)

#sélection des objets pertinents
est2<-coef(lm2)[2]
c2<-confint(lm2,level=0.95)[2,]

#création d'un dataframe
est2 <- data.frame(est = est2,
                   c_inf = c2[1],
                   c_sup = c2[2], 
                   model = "Modèle multiple")

#voir le dataframe crée:
est2

In [ ]:
#graphique
ggplot(est2,aes(x=model,y=est)) +
    geom_point() +
    geom_errorbar(aes(ymin=c_inf,ymax=c_sup,width=0.1)) +
    geom_hline(yintercept=0, color="white") +
    xlab("") +
    ylab("Opinion des femmes sur le parti démocrate (relativement aux hommes)")

Note: 

- Si l'on a bien nommé les éléments des dataframes est1 et est2 de la même manière, à savoir par exemple ici: est, c_inf, c_sup et model, 
- Alors on pourra facilement représenter les deux estimations sur un même graphique à titre de comparaison, 
- En utilisant **rbind()** pour ajouter les lignes (row) des dataframes *est1* et *est2*
- Ce qui donne un seul dataframe combiné :

In [ ]:
est<-rbind(est1,est2)
est

In [ ]:
ggplot(est,aes(x=model, y=est)) + 
    geom_point() +
    geom_errorbar(aes(ymin=c_inf,ymax=c_sup), width=0.1) +
    geom_hline(yintercept = 0, color="white") +
    xlab("") +
    ylab("Opinion des femmes sur le parti démocrate (relativement aux hommes)")

### 2.3 Interactions
On peut souhaiter savoir si l'effet du genre sur le score pro-démocrate varie selon l'âge de la personne. 

Pour ce faire, il est nécessaire d'introduire un terme d'interaction dans la régression, entre l'âge et le genre. 

On estime le modèle suivant: 

In [ ]:
lm3 <- lm(demtherm~gender+age+income+age:gender, data = anes)

In [ ]:
summary(lm3)

Discussion: interpréter.

On peut également imaginer représenter l'effet différencié du genre selon l'âge de la personne par un graphique : 

In [ ]:
#on crée les catégories:
predict_for <- data.frame(expand.grid(gender = c("Homme", "Femme"),
                                 age = 18:85), 
                     income = "3. 34 to 67 percentile")

In [ ]:
#on prédit les valeurs du score pro-démocrate correspondant à ces catégories
pred3 <- predict(lm3,
             newdata=predict_for,
             se.fit=T,
             interval="confidence")

In [ ]:
#pour voir:
head(predict_for)
head(pred3)

In [ ]:
#on combine les colonnes de pred3 et predict_for en un seul dataframe pred3:
pred3 <- data.frame(cbind(pred3$fit,predict_for))
head(pred3)

In [ ]:
#on utilise ce dataframe pour créer le graphique:
ggplot(pred3, aes(x=age,y=fit,color=gender)) +
    geom_line() + geom_point() +
    geom_line(aes(y=lwr),lty=3) + geom_line(aes(y=upr),lty=3)

Discussion: conclure sur le graphique.

On sauve les données avec **saveRDS()** :

In [ ]:
# Save an object to a file
saveRDS(anes, file = "TD4_mydata.rds")

Note: Pour ouvrir un objet dans R : 
**readRDS(file = "TD4_my_data.rds")**

### Exercice 2
On souhaite connaître l'impact de l'ethnie sur le score pro-démocrate. On estime un modèle où le score *demtherm* dépend: du genre *gender*, de l'âge *age*, du niveau d'éducation *edu*, de l'ethnie *race*. Le codebook nous apprend que *race* prend 3 valeurs: 1 si 'white', 2 si 'black', 3 si 'other', 9 si 'missing'.

Commencez par restaurer l'objet *"TD4_mydata.rds"*. Créez un dataframequi contient uniquement les données qui nous intéressent (rappel: TD2: **library(dplyr)**, piping **%>%**, fonction **select()**).

In [ ]:
library(plyr)
library(dplyr)
d<-readRDS(file="TD4_mydata.rds")
d<-d%>%dplyr::select(demtherm,age,gender,edu,race)

Produisez des statistiques descriptives basiques sur ces variables (summary()).

In [ ]:
summary(d)

Créez deux variables : 
- *white* qui vaudra 1 si l'individu se définit comme blanc américain, 0 s'il se définit comme noir américain ou autre
- *edu3* qui regroupera 1. et 2. en 'Pre-bac', 3. et 4. en 'Bac', 5 et 6. en 'Université', 8. et 9 en NA

In [ ]:
#white
d$white<-ifelse(d$race==1,1,0) #1 si blanc, 0 sinon
d$white[d$race==9]<-NA  #attention à bien coder les variables manquantes en 'NA'
table(d$white, exclude=NULL) #on vérifie
d$white<-as.factor(d$white) #on définit cette variable comme facteur
levels(d$white)<-c("Non-White","White")

#edu3
summary(d$edu)
levels(d$edu)
d$edu3<-d$edu
levels(d$edu3)<-c("Pre-bac","Pre-bac","Bac","Bac","Université","Université",NA,NA)
levels(d$edu3)  #on vérifie
summary(d$edu3)

Effectuer la régression de *demtherm* sur *age*, *genre*, et les deux variables nouvellement crées: *edu3* et *white*. Commenter.

In [ ]:
lm1<-lm(demtherm~gender+age+edu3+white, data=d)
summary(lm1)

Représenter graphiquement le score moyen des blancs américains et non-blancs (on prendra les hommes de niveau bac et d'âge de 40).

In [ ]:
#catégorie pour laquelle on va prédire :
predict_for <- data.frame(expand.grid(gender = "Homme",
                                     age = 40, 
                                     edu3 = "Bac",
                                     white = c("Non-White","White")))
predict_for

#on prédit :
pred1<-predict(lm1,
             newdata=predict_for,
             se.fit=T,
             interval="confidence"
             )
pred1

#on associe:
pred1 <- data.frame(cbind(pred1$fit,predict_for))
pred1

#graphique :
library(ggplot2)
ggplot(pred1, aes(x=white,y=fit)) + geom_point() + geom_errorbar(aes(ymin=lwr,ymax=upr), width=0.1)  + xlab("") + ylab("Opinion pro-démocrate")

Refaire l'estimation en interagissant l'ethnie avec l'éducation :

In [ ]:
lm2 <- lm(demtherm~gender+age+edu3+white+white:edu3, data=d)
summary(lm2)

OPTIONNEL : Représenter graphiquement les scores moyens des hommes de 40 ans selon leur niveau d'étude, pour les non-whites et whites (les scores moyens avec leu rintervalle de confiance en ordonnée, les niveaux d'éducation en absisse, et les origines ethniques en légende).

In [ ]:
#catégories pour lesquelles on prédit:
predict_for<-data.frame(expand.grid(gender="Homme",
                                   age=40,
                                   white=c("Non-White","White"),
                                   edu3=c("Pre-bac","Bac","Université")))
predict_for

#valeurs prédites:
pred2<-predict(lm2,
              newdata=predict_for,
              se.fit=T,
              interval="confidence")
pred2

#combinaison des deux objets en 1 dataframe
pred2<-data.frame(cbind(pred2$fit,predict_for))
pred2

#on utilise ce dataframe pour créer le graphique:
ggplot(pred2, aes(x=edu3,y=fit,color=white)) +
    geom_point() +
    geom_errorbar(aes(ymin=lwr,ymax=upr),width=0.1) 

Conclure.